In [ ]:
!pip install traveltimepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import traveltimepy as ttpy
import os
from datetime import datetime #for examples
#store your credentials in an environment variable
os.environ["TRAVELTIME_ID"] = '30fd4fd0'
os.environ["TRAVELTIME_KEY"] = 'ad1b4423f781df34f34b449c483b640a'

from tabulate import tabulate
import pandas as pd
import numpy as np

In [ ]:
# Import Hospital Coordinates Dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
hospitals = pd.read_csv("/content/drive/MyDrive/Optimization Project/Data/hospitals_sub.csv")

In [ ]:
#Selecting only the relevant columns
#data = data[['Name', 'Latitude', 'Longitude']]
pd.DataFrame.head(hospitals)

,Index,Name,Latitude,Longitude
0,1,Arbour Hospital,42.315770,-71.112250
1,2,Beth Israel Deaconess Medical Center - West Ca...,42.337594,-71.108921
2,3,Beth Isreal Deaconess medical Center - East Ca...,42.339820,-71.105680
3,4,Boston Medical Center - East Newton Campus,42.337759,-71.070795
4,5,Boston Medical Center - Menino Campus,42.334784,-71.073204


In [ ]:
depots = pd.read_csv("/content/drive/MyDrive/Optimization Project/Data/depots.csv")
pd.DataFrame.head(depots)

,Index,Name,Latitude,Longitude
0,1,Depot 1,42.38106,-71.08862
1,2,Depot 2,42.32061,-71.09926


In [ ]:
pt_map = pd.read_csv("/content/drive/MyDrive/Optimization Project/Data/pt_map.csv")

In [ ]:
pd.DataFrame.head(pt_map)

,Patient Index,Origin Hospital,Destination Hospital
0,1,2,6
1,2,2,8
2,3,2,13
3,4,2,17
4,5,3,6


In [ ]:
data = depots

pd.DataFrame.head(data)

#Obtaining the number of depots
num_depots = depots.shape[0]
type(num_depots)

#Convert index column to int so we can adjust it
#hospitals_adjusted = hospitals.apply(lambda x: int(x) if x.column == 'Index' else x)

hospitals["Index"] = hospitals["Index"].astype(int)

type(hospitals["Index"][0])

#Adjusting the hospital indices so that they are d+1:h+d rows
hospitals_adjusted = hospitals.apply(lambda x: (x+num_depots) if x.name == 'Index' else x)

In [ ]:
#Obtaining the number of hospitals
num_hospitals = hospitals.shape[0]
num_hospitals

24

In [ ]:
#Concatenating the depots and adjusted hospitals together
data = pd.concat([data, hospitals_adjusted], axis=0, ignore_index = True)

pd.DataFrame.head(data, 50)

,Index,Name,Latitude,Longitude
0,1,Depot 1,42.381060,-71.088620
1,2,Depot 2,42.320610,-71.099260
2,3,Arbour Hospital,42.315770,-71.112250
3,4,Beth Israel Deaconess Medical Center - West Ca...,42.337594,-71.108921
4,5,Beth Isreal Deaconess medical Center - East Ca...,42.339820,-71.105680
5,6,Boston Medical Center - East Newton Campus,42.337759,-71.070795
6,7,Boston Medical Center - Menino Campus,42.334784,-71.073204
7,8,Brigham and Women's Hospital,42.336106,-71.107480
8,9,Carney Hospital,42.277426,-71.065693
9,10,Children's Hospital Boston,42.336920,-71.105420


In [ ]:
#Creating a list of dictionaries call locations
locations_list = []

#Appending dictionary elements
for i in range(data['Name'].size):
  locations_list.append({"id": data['Name'][i], "coords": {"lat": data['Latitude'][i], "lng": data["Longitude"][i]}})

In [ ]:
departure_searches = []
outs = []

for i in range(len(locations_list)):
  departure_searches.append({
    "id": "forward search example",
    "departure_location_id": locations_list[i]['id'],
    "arrival_location_ids": [locations_list[j]['id'] for j in range(len(locations_list))],
    "transportation": {"type": "driving"},
    "departure_time":  datetime.utcnow().isoformat(),
    "travel_time": 3600,
    "properties": ["distance"],
    "range": {"enabled": False, "max_results": 5, "width": 10000}
  })

  outs.append(ttpy.time_filter(
    locations=locations_list, departure_searches=departure_searches[i]))

In [ ]:
for i in range(len(locations_list)):
  print(locations_list[i]['id'])

Depot 1
Depot 2
Arbour Hospital
Beth Israel Deaconess Medical Center - West Campus
Beth Isreal Deaconess medical Center - East Campus
Boston Medical Center - East Newton Campus
Boston Medical Center - Menino Campus
Brigham and Women's Hospital
Carney Hospital
Children's Hospital Boston
Dana-Farber Cancer Institute
Erich LIindemann Mental Health Center
Faulkner Hospital
Franciscan Children's Hospital & Rehab Center
Hebrew Rehabilitation Center
Jewish Memorial Hospital & Radius Hospital
Kindred Hospital
Lemuel Shattuck Hospital
Mass General Hospital
Massachusetts Eye and Ear Infirmary
New England Baptist Hospital
Shriner’s Burn Institute 
Spaulding Rehabilitation Hospital
St. Elizabeth's Hospital
Tufts Medical Center
VA Bos. Healthcare System - W. Roxbury


In [ ]:
outs

In [ ]:
#Creating a list of destinations and their respective distances
destinations_list = []
distances_list = []

for i in range(len(locations_list)):
  for j in range(len(locations_list)):
    dest = []
    distDict = []

    for l in range(len(locations_list)):
      #Output from one departure location to many destinations
      items = outs[i]['results'][0]['locations'][l].items()
      for k, v in items:
        #Values just ignores the labels like 'id' and 'properties'
        if (k == 'id'):
          #This is adding the destination name to the list of destination names
          dest.append(v)

        if (k == 'properties'):
          #This is adding the distance to the list of distances
          distDict.append(v[0]['distance'])

  destinations_list.append(dest)
  distances_list.append(distDict)

print(destinations_list)
print(distances_list)

[['Faulkner Hospital', 'Massachusetts Eye and Ear Infirmary', 'Lemuel Shattuck Hospital', 'Spaulding Rehabilitation Hospital', 'Depot 2', "Children's Hospital Boston", 'Kindred Hospital', "Brigham and Women's Hospital", 'Mass General Hospital', 'Tufts Medical Center', 'Depot 1', 'Hebrew Rehabilitation Center', 'VA Bos. Healthcare System - W. Roxbury', 'Beth Israel Deaconess Medical Center - West Campus', 'Beth Isreal Deaconess medical Center - East Campus', 'Erich LIindemann Mental Health Center', 'Dana-Farber Cancer Institute', 'Jewish Memorial Hospital & Radius Hospital', 'New England Baptist Hospital', "St. Elizabeth's Hospital", 'Shriner’s Burn Institute\xa0', "Franciscan Children's Hospital & Rehab Center", 'Boston Medical Center - Menino Campus', 'Carney Hospital', 'Arbour Hospital', 'Boston Medical Center - East Newton Campus'], ['Faulkner Hospital', 'Massachusetts Eye and Ear Infirmary', 'Lemuel Shattuck Hospital', 'Spaulding Rehabilitation Hospital', 'Depot 2', "Children's Hos

In [ ]:
#Creating a list of data frames which we will later concatenate
df_list = []

# Calling DataFrame constructor on list with indices and columns specified
for i in range(len(locations_list)):
  df_list.append(pd.DataFrame(distances_list[i], index = destinations_list[i], columns = [locations_list[i]['id']]))



In [ ]:
#Concatenating Data Frames
df = pd.concat([df_list[i] for i in range(len(df_list))], axis=1)
df

,Depot 1,Depot 2,Arbour Hospital,Beth Israel Deaconess Medical Center - West Campus,Beth Isreal Deaconess medical Center - East Campus,Boston Medical Center - East Newton Campus,Boston Medical Center - Menino Campus,Brigham and Women's Hospital,Carney Hospital,Children's Hospital Boston,...,Kindred Hospital,Lemuel Shattuck Hospital,Mass General Hospital,Massachusetts Eye and Ear Infirmary,New England Baptist Hospital,Shriner’s Burn Institute,Spaulding Rehabilitation Hospital,St. Elizabeth's Hospital,Tufts Medical Center,VA Bos. Healthcare System - W. Roxbury
Faulkner Hospital,16198,4102,3284,5263,5675,7525,7466,5409,8474,5777,...,7697,3542,10621,11574,5444,10878,11025,8520,9797,8224
Massachusetts Eye and Ear Infirmary,6809,6915,8628,5778,4864,6530,7381,5590,14432,5454,...,12991,12439,255,0,6284,551,1636,12245,4887,37054
Lemuel Shattuck Hospital,16714,3601,3828,6175,6595,6504,6227,5929,5166,6455,...,8539,0,12932,12743,5488,12824,12583,9362,9344,8446
Spaulding Rehabilitation Hospital,5829,9568,10822,6602,5688,5592,6443,6414,13494,6278,...,12053,11501,1381,1192,7108,1023,0,11868,3949,36116
Depot 2,13531,0,2106,2738,3158,3938,3981,2492,7060,3018,...,6715,3790,9749,9560,2051,9641,9400,6910,6161,12272
Children's Hospital Boston,11069,3481,3624,534,546,4349,4392,590,10534,0,...,5041,7212,5492,6445,1579,5749,5896,4982,6572,11801
Kindred Hospital,16716,7381,6648,4322,4734,11241,11363,4468,19112,4836,...,0,8330,12934,12745,5479,12826,12585,1048,9342,11186
Brigham and Women's Hospital,11253,2955,3101,310,730,3951,3994,0,10008,590,...,4676,6689,5676,6629,1053,5933,6080,4617,6174,11278
Mass General Hospital,6729,6813,8526,5676,4762,6450,7301,5488,14352,5352,...,12911,12359,0,895,6182,451,1556,12143,4807,32786
Tufts Medical Center,8055,6797,8051,6654,8580,2828,2950,6406,10525,9170,...,9294,10205,4273,4084,7192,4165,3924,9109,0,33357


In [ ]:
df

,Depot 1,Depot 2,Arbour Hospital,Beth Israel Deaconess Medical Center - West Campus,Beth Isreal Deaconess medical Center - East Campus,Boston Medical Center - East Newton Campus,Boston Medical Center - Menino Campus,Brigham and Women's Hospital,Carney Hospital,Children's Hospital Boston,...,Kindred Hospital,Lemuel Shattuck Hospital,Mass General Hospital,Massachusetts Eye and Ear Infirmary,New England Baptist Hospital,Shriner’s Burn Institute,Spaulding Rehabilitation Hospital,St. Elizabeth's Hospital,Tufts Medical Center,VA Bos. Healthcare System - W. Roxbury
Faulkner Hospital,16198,4102,3284,5263,5675,7525,7466,5409,8474,5777,...,7697,3542,10621,11574,5444,10878,11025,8520,9797,8224
Massachusetts Eye and Ear Infirmary,6809,6915,8628,5778,4864,6530,7381,5590,14432,5454,...,12991,12439,255,0,6284,551,1636,12245,4887,37054
Lemuel Shattuck Hospital,16714,3601,3828,6175,6595,6504,6227,5929,5166,6455,...,8539,0,12932,12743,5488,12824,12583,9362,9344,8446
Spaulding Rehabilitation Hospital,5829,9568,10822,6602,5688,5592,6443,6414,13494,6278,...,12053,11501,1381,1192,7108,1023,0,11868,3949,36116
Depot 2,13531,0,2106,2738,3158,3938,3981,2492,7060,3018,...,6715,3790,9749,9560,2051,9641,9400,6910,6161,12272
Children's Hospital Boston,11069,3481,3624,534,546,4349,4392,590,10534,0,...,5041,7212,5492,6445,1579,5749,5896,4982,6572,11801
Kindred Hospital,16716,7381,6648,4322,4734,11241,11363,4468,19112,4836,...,0,8330,12934,12745,5479,12826,12585,1048,9342,11186
Brigham and Women's Hospital,11253,2955,3101,310,730,3951,3994,0,10008,590,...,4676,6689,5676,6629,1053,5933,6080,4617,6174,11278
Mass General Hospital,6729,6813,8526,5676,4762,6450,7301,5488,14352,5352,...,12911,12359,0,895,6182,451,1556,12143,4807,32786
Tufts Medical Center,8055,6797,8051,6654,8580,2828,2950,6406,10525,9170,...,9294,10205,4273,4084,7192,4165,3924,9109,0,33357


In [ ]:
#Changing Depots names to _Depot 1 and _Depot 2 so they come to the front of the data matrix
#I would do this in the CSV that I import, but I'm doing this post facto

df = df.rename({'_Depot 1': '%Depot 1', '_Depot 2': '%Depot 2'}, axis=1)

df = df.rename({'Depot 1': '%Depot 1', 'Depot 2': '%Depot 2'}, axis=0)
df

,%Depot 1,%Depot 2,Arbour Hospital,Beth Israel Deaconess Medical Center - West Campus,Beth Isreal Deaconess medical Center - East Campus,Boston Medical Center - East Newton Campus,Boston Medical Center - Menino Campus,Brigham and Women's Hospital,Carney Hospital,Children's Hospital Boston,...,Kindred Hospital,Lemuel Shattuck Hospital,Mass General Hospital,Massachusetts Eye and Ear Infirmary,New England Baptist Hospital,Shriner’s Burn Institute,Spaulding Rehabilitation Hospital,St. Elizabeth's Hospital,Tufts Medical Center,VA Bos. Healthcare System - W. Roxbury
Arbour Hospital,15052,1777,0,3783,4195,5504,5547,3929,8310,4297,...,6249,3378,11270,11081,3182,11162,10921,7072,7682,10606
Beth Israel Deaconess Medical Center - West Campus,10963,3200,3085,0,440,4199,4242,310,10253,534,...,4538,6673,5386,6339,1298,5643,5790,4479,6422,11262
Beth Isreal Deaconess medical Center - East Campus,10618,3628,3504,447,0,4626,4669,737,10681,553,...,4957,7092,5041,5994,1726,5298,5445,5091,5202,11681
Boston Medical Center - East Newton Campus,10891,4501,5755,4358,4778,0,835,4110,9989,4368,...,11724,6953,7109,6920,4896,7001,6760,11539,3521,35787
Boston Medical Center - Menino Campus,10551,3892,5146,3749,4169,641,0,3501,9649,3759,...,11384,6533,6769,6580,4287,6661,6420,11199,3181,14787
Brigham and Women's Hospital,11253,2955,3101,310,730,3951,3994,0,10008,590,...,4676,6689,5676,6629,1053,5933,6080,4617,6174,11278
Carney Hospital,18868,7703,8255,13761,14181,10894,10720,13513,0,13771,...,19701,5829,15086,14897,9590,14978,14737,19516,11498,14237
Children's Hospital Boston,11069,3481,3624,534,546,4349,4392,590,10534,0,...,5041,7212,5492,6445,1579,5749,5896,4982,6572,11801
Dana-Farber Cancer Institute,11033,3173,3316,226,510,4171,4214,282,10226,364,...,4733,6904,5456,6409,1271,5713,5860,4674,6394,11493
%Depot 1,0,12165,13419,8451,7537,8189,9040,8263,16091,8127,...,14650,14098,3230,3041,12560,3122,2860,14465,6546,38713


In [ ]:
#Sort rows based on index labels
df = df.sort_index()
df

,Depot 1,Depot 2,Arbour Hospital,Beth Israel Deaconess Medical Center - West Campus,Beth Isreal Deaconess medical Center - East Campus,Boston Medical Center - East Newton Campus,Boston Medical Center - Menino Campus,Brigham and Women's Hospital,Carney Hospital,Children's Hospital Boston,...,Kindred Hospital,Lemuel Shattuck Hospital,Mass General Hospital,Massachusetts Eye and Ear Infirmary,New England Baptist Hospital,Shriner’s Burn Institute,Spaulding Rehabilitation Hospital,St. Elizabeth's Hospital,Tufts Medical Center,VA Bos. Healthcare System - W. Roxbury
Arbour Hospital,15052,1777,0,3783,4195,5504,5547,3929,8310,4297,...,6249,3378,11270,11081,3182,11162,10921,7072,7682,10606
Beth Israel Deaconess Medical Center - West Campus,10963,3200,3085,0,440,4199,4242,310,10253,534,...,4538,6673,5386,6339,1298,5643,5790,4479,6422,11262
Beth Isreal Deaconess medical Center - East Campus,10618,3628,3504,447,0,4626,4669,737,10681,553,...,4957,7092,5041,5994,1726,5298,5445,5091,5202,11681
Boston Medical Center - East Newton Campus,10891,4501,5755,4358,4778,0,835,4110,9989,4368,...,11724,6953,7109,6920,4896,7001,6760,11539,3521,35787
Boston Medical Center - Menino Campus,10551,3892,5146,3749,4169,641,0,3501,9649,3759,...,11384,6533,6769,6580,4287,6661,6420,11199,3181,14787
Brigham and Women's Hospital,11253,2955,3101,310,730,3951,3994,0,10008,590,...,4676,6689,5676,6629,1053,5933,6080,4617,6174,11278
Carney Hospital,18868,7703,8255,13761,14181,10894,10720,13513,0,13771,...,19701,5829,15086,14897,9590,14978,14737,19516,11498,14237
Children's Hospital Boston,11069,3481,3624,534,546,4349,4392,590,10534,0,...,5041,7212,5492,6445,1579,5749,5896,4982,6572,11801
Dana-Farber Cancer Institute,11033,3173,3316,226,510,4171,4214,282,10226,364,...,4733,6904,5456,6409,1271,5713,5860,4674,6394,11493
Depot 1,0,12165,13419,8451,7537,8189,9040,8263,16091,8127,...,14650,14098,3230,3041,12560,3122,2860,14465,6546,38713


In [ ]:
#Changing Depots names back to Depot 1 and Depot 2

df = df.rename({'%Depot 1': 'Depot 1', '%Depot 2': 'Depot 2'}, axis=1)

df = df.rename({'%Depot 1': 'Depot 1', '%Depot 2': 'Depot 2'}, axis=0)
df


In [ ]:
#Save unsorted data frame
df.to_csv("/content/drive/MyDrive/Optimization Project/Data/UnsortedDistances_sub.csv", index=True, encoding='utf-8-sig')

In [ ]:
#Sort rows based on index labels
df = df.sort_index()

#Sort columns based on column labels
df = df.sort_index(axis = 1)

df

,%Depot 1,%Depot 2,Arbour Hospital,Beth Israel Deaconess Medical Center - West Campus,Beth Isreal Deaconess medical Center - East Campus,Boston Medical Center - East Newton Campus,Boston Medical Center - Menino Campus,Brigham and Women's Hospital,Carney Hospital,Children's Hospital Boston,...,Kindred Hospital,Lemuel Shattuck Hospital,Mass General Hospital,Massachusetts Eye and Ear Infirmary,New England Baptist Hospital,Shriner’s Burn Institute,Spaulding Rehabilitation Hospital,St. Elizabeth's Hospital,Tufts Medical Center,VA Bos. Healthcare System - W. Roxbury
%Depot 1,0,12165,13419,8451,7537,8189,9040,8263,16091,8127,...,14650,14098,3230,3041,12560,3122,2860,14465,6546,38713
%Depot 2,13531,0,2106,2738,3158,3938,3981,2492,7060,3018,...,6715,3790,9749,9560,2051,9641,9400,6910,6161,12272
Arbour Hospital,15052,1777,0,3783,4195,5504,5547,3929,8310,4297,...,6249,3378,11270,11081,3182,11162,10921,7072,7682,10606
Beth Israel Deaconess Medical Center - West Campus,10963,3200,3085,0,440,4199,4242,310,10253,534,...,4538,6673,5386,6339,1298,5643,5790,4479,6422,11262
Beth Isreal Deaconess medical Center - East Campus,10618,3628,3504,447,0,4626,4669,737,10681,553,...,4957,7092,5041,5994,1726,5298,5445,5091,5202,11681
Boston Medical Center - East Newton Campus,10891,4501,5755,4358,4778,0,835,4110,9989,4368,...,11724,6953,7109,6920,4896,7001,6760,11539,3521,35787
Boston Medical Center - Menino Campus,10551,3892,5146,3749,4169,641,0,3501,9649,3759,...,11384,6533,6769,6580,4287,6661,6420,11199,3181,14787
Brigham and Women's Hospital,11253,2955,3101,310,730,3951,3994,0,10008,590,...,4676,6689,5676,6629,1053,5933,6080,4617,6174,11278
Carney Hospital,18868,7703,8255,13761,14181,10894,10720,13513,0,13771,...,19701,5829,15086,14897,9590,14978,14737,19516,11498,14237
Children's Hospital Boston,11069,3481,3624,534,546,4349,4392,590,10534,0,...,5041,7212,5492,6445,1579,5749,5896,4982,6572,11801


In [ ]:
#Save sorted data frame
df.to_csv("/content/drive/MyDrive/Optimization Project/Data/SortedDistances_sub2.csv", index=True, encoding='utf-8-sig')

In [ ]:
#Repeating the same process but for time, not distance
departure_searches_time = []
outs_time = []

for i in range(len(locations_list)):
  departure_searches_time.append({
    "id": "forward search example",
    "departure_location_id": locations_list[i]['id'],
    "arrival_location_ids": [locations_list[j]['id'] for j in range(len(locations_list))],
    "transportation": {"type": "driving"},
    "departure_time":  datetime.utcnow().isoformat(),
    "travel_time": 3600,
    "properties": ["travel_time"],
    "range": {"enabled": False, "max_results": 5, "width": 10000}
  })

  outs_time.append(ttpy.time_filter(
    locations=locations_list, departure_searches=departure_searches_time[i]))

In [ ]:
#Creating a list of destinations and their respective travel times
destinations_list_time = []
time_list = []

for i in range(len(locations_list)):
  for j in range(len(locations_list)):
    dest = []
    distDict = []

    for l in range(len(locations_list)):
      #Output from one departure location to many destinations
      items = outs_time[i]['results'][0]['locations'][l].items()
      for k, v in items:
        #Values just ignores the labels like 'id' and 'properties'
        if (k == 'id'):
          #This is adding the destination name to the list of destination names
          dest.append(v)

        if (k == 'properties'):
          #This is adding the distance to the list of distances
          distDict.append(v[0]['travel_time'])

  destinations_list_time.append(dest)
  time_list.append(distDict)

print(destinations_list_time)
print(time_list)

[['Faulkner Hospital', 'Massachusetts Eye and Ear Infirmary', 'Lemuel Shattuck Hospital', 'Spaulding Rehabilitation Hospital', 'Depot 2', "Children's Hospital Boston", 'Kindred Hospital', "Brigham and Women's Hospital", 'Mass General Hospital', 'Tufts Medical Center', 'Depot 1', 'Hebrew Rehabilitation Center', 'VA Bos. Healthcare System - W. Roxbury', 'Beth Israel Deaconess Medical Center - West Campus', 'Beth Isreal Deaconess medical Center - East Campus', 'Erich LIindemann Mental Health Center', 'Dana-Farber Cancer Institute', 'Jewish Memorial Hospital & Radius Hospital', 'New England Baptist Hospital', "St. Elizabeth's Hospital", 'Shriner’s Burn Institute\xa0', "Franciscan Children's Hospital & Rehab Center", 'Boston Medical Center - Menino Campus', 'Carney Hospital', 'Arbour Hospital', 'Boston Medical Center - East Newton Campus'], ['Faulkner Hospital', 'Massachusetts Eye and Ear Infirmary', 'Lemuel Shattuck Hospital', 'Spaulding Rehabilitation Hospital', 'Depot 2', "Children's Hos

In [ ]:
#Creating a list of data frames which we will later concatenate
df_list_time = []

# Calling DataFrame constructor on list with indices and columns specified
for i in range(len(locations_list)):
  df_list_time.append(pd.DataFrame(time_list[i], index = destinations_list_time[i], columns = [locations_list[i]['id']]))

In [ ]:
#Concatenating Data Frames
df_time = pd.concat([df_list_time[i] for i in range(len(df_list_time))], axis=1)

In [ ]:
df_time = df_time.rename({'Depot 1': '%Depot 1', 'Depot 2': '%Depot 2'}, axis=1)

df_time = df_time.rename({'Depot 1': '%Depot 1', 'Depot 2': '%Depot 2'}, axis=0)
df_time

,%Depot 1,%Depot 2,Arbour Hospital,Beth Israel Deaconess Medical Center - West Campus,Beth Isreal Deaconess medical Center - East Campus,Boston Medical Center - East Newton Campus,Boston Medical Center - Menino Campus,Brigham and Women's Hospital,Carney Hospital,Children's Hospital Boston,...,Kindred Hospital,Lemuel Shattuck Hospital,Mass General Hospital,Massachusetts Eye and Ear Infirmary,New England Baptist Hospital,Shriner’s Burn Institute,Spaulding Rehabilitation Hospital,St. Elizabeth's Hospital,Tufts Medical Center,VA Bos. Healthcare System - W. Roxbury
Faulkner Hospital,1827,885,563,725,808,1571,1494,749,1227,840,...,974,688,1552,1606,828,1609,1522,1072,1653,854
Massachusetts Eye and Ear Infirmary,697,973,1222,845,866,728,729,822,1040,827,...,1180,1332,104,0,930,230,392,1156,690,1941
Lemuel Shattuck Hospital,1481,629,734,951,1024,1127,1028,923,632,1028,...,1420,0,1207,1147,885,1229,1170,1518,1165,1435
Spaulding Rehabilitation Hospital,556,893,1184,899,920,588,589,876,900,881,...,1040,1192,342,282,984,338,0,1020,550,1801
%Depot 2,1203,0,405,397,470,779,705,369,1017,474,...,1014,671,929,869,331,951,892,1110,887,1412
Children's Hospital Boston,1178,508,529,137,159,959,885,139,1382,0,...,800,1032,903,957,285,960,873,877,1067,1286
Kindred Hospital,1416,1034,994,673,756,1231,1333,697,1535,788,...,0,1314,1142,1082,811,1164,1105,215,1075,1337
Brigham and Women's Hospital,1173,420,441,73,154,892,818,0,1294,158,...,732,944,898,952,197,955,868,809,1000,1198
Mass General Hospital,715,985,1234,857,878,746,747,834,1058,839,...,1198,1350,0,325,942,226,410,1168,708,1959
Tufts Medical Center,891,1085,1376,1171,1195,674,776,1139,927,1156,...,1116,1263,617,557,1194,639,580,1096,0,1877


In [ ]:
#Sort rows based on index labels
df_time = df_time.sort_index()

#Sort columns based on column labels
df_time = df_time.sort_index(axis = 1)

In [ ]:
df_time

,%Depot 1,%Depot 2,Arbour Hospital,Beth Israel Deaconess Medical Center - West Campus,Beth Isreal Deaconess medical Center - East Campus,Boston Medical Center - East Newton Campus,Boston Medical Center - Menino Campus,Brigham and Women's Hospital,Carney Hospital,Children's Hospital Boston,...,Kindred Hospital,Lemuel Shattuck Hospital,Mass General Hospital,Massachusetts Eye and Ear Infirmary,New England Baptist Hospital,Shriner’s Burn Institute,Spaulding Rehabilitation Hospital,St. Elizabeth's Hospital,Tufts Medical Center,VA Bos. Healthcare System - W. Roxbury
%Depot 1,0,1117,1408,1179,1200,812,813,1156,1124,1161,...,1264,1416,622,562,1226,644,560,1244,774,2025
%Depot 2,1203,0,405,397,470,779,705,369,1017,474,...,1014,671,929,869,331,951,892,1110,887,1412
Arbour Hospital,1552,404,0,530,613,1129,1055,554,1194,645,...,1078,655,1278,1218,552,1300,1241,1176,1236,1201
Beth Israel Deaconess Medical Center - West Campus,1119,437,437,0,100,914,840,64,1311,138,...,702,940,844,898,214,901,814,779,1022,1194
Beth Isreal Deaconess medical Center - East Campus,1075,508,521,104,0,989,915,139,1382,162,...,786,1024,800,854,285,857,770,853,1042,1278
Boston Medical Center - East Newton Campus,952,796,1087,882,963,0,286,850,1004,934,...,1225,1164,678,618,905,700,641,1205,636,1986
Boston Medical Center - Menino Campus,867,681,972,767,848,263,0,735,919,819,...,1140,1056,593,533,790,615,556,1120,551,1889
Brigham and Women's Hospital,1173,420,441,73,154,892,818,0,1294,158,...,732,944,898,952,197,955,868,809,1000,1198
Carney Hospital,1238,1031,1233,1322,1403,952,965,1290,0,1374,...,1511,755,964,904,1287,986,927,1491,922,1763
Children's Hospital Boston,1178,508,529,137,159,959,885,139,1382,0,...,800,1032,903,957,285,960,873,877,1067,1286


In [ ]:
df_time = df_time.rename({'%Depot 1': 'Depot 1', '%Depot 2': 'Depot 2'}, axis=1)

df_time = df_time.rename({'%Depot 1': 'Depot 1', '%Depot 2': 'Depot 2'}, axis=0)
df_time

In [ ]:
#Save sorted data frame
df_time.to_csv("/content/drive/MyDrive/Optimization Project/Data/SortedTimes_sub2.csv", index=True, encoding='utf-8-sig')